In [7]:
!pip install mlflow

In [8]:
!mlflow --version

mlflow, version 2.19.0


In [9]:
# Step-1 Install requirements
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import mlflow
import mlflow.sklearn

In [10]:
# Step-2 Load Data set
data = pd.read_csv('C:/Users/igor/Downloads/Data-Sets-ML/winequality-white.csv',delimiter=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [11]:
data.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [11]:
# split the train and test divide data into 70 and 30 percent and randomly
train,test= train_test_split(data,test_size=0.3,random_state=1234)
train.shape,test.shape

((3428, 12), (1470, 12))

In [12]:
train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1605,7.1,0.26,0.49,2.2,0.032,31.0,113.0,0.99030,3.37,0.42,12.9,9
1121,6.0,0.40,0.30,1.6,0.047,30.0,117.0,0.99310,3.17,0.48,10.1,6
818,6.2,0.30,0.17,2.8,0.040,24.0,125.0,0.99390,3.01,0.46,9.0,5
2872,4.2,0.17,0.36,1.8,0.029,93.0,161.0,0.98999,3.65,0.89,12.0,7
232,8.2,0.27,0.39,7.8,0.039,49.0,208.0,0.99760,3.31,0.51,9.5,6


In [13]:
# Now divide data into train_x and train_y
train_x=train.drop(['quality'], axis=1)
test_x =test.drop(['quality'],axis=1)

train_y=train[['quality']]
test_y=test[['quality']]

In [16]:
train_x.shape,train_y.shape

((3428, 11), (3428, 1))

In [17]:
test_x.shape,test_y.shape

((1470, 11), (1470, 1))

In [14]:
# Step-5 Start mlflow by creating workspace(experiment)
mlflow.set_experiment('/mlflow/igor_workspace')

<Experiment: artifact_location='file:///C:/Users/igor/mlopssir/mlruns/222810493346004295', creation_time=1735785488795, experiment_id='222810493346004295', last_update_time=1735785488795, lifecycle_stage='active', name='/mlflow/igor_workspace', tags={}>

In [15]:
print(mlflow.set_experiment('/mlflow/igor_workspace').experiment_id)
print(mlflow.set_experiment('/mlflow/igor_workspace').lifecycle_stage)
print(mlflow.set_experiment('/mlflow/igor_workspace').name)

222810493346004295
active
/mlflow/igor_workspace


In [16]:
# Train the model
def train_model(alpha,l1_ratio):
  train,test= train_test_split(data,test_size=0.3,random_state=1234)
  train_x=train.drop(['quality'], axis=1)
  test_x =test.drop(['quality'],axis=1)
  train_y=train[['quality']]
  test_y=test[['quality']]

# Initiate mlflow now(run)
  with mlflow.start_run(experiment_id=222810493346004295,run_name="regression",description="performing analysis"):
    # Call ML model, variable lr
    lr=ElasticNet(alpha=alpha,l1_ratio=l1_ratio)
    lr.fit(train_x,train_y)

    # call model prediction on test_x data
    predicted_data=lr.predict(test_x)

    # Model evaluation(compare test data vs prediction data)
    rmse=np.sqrt(mean_squared_error(test_y,predicted_data))
    mae=mean_absolute_error(test_y,predicted_data)
    r2=r2_score(test_y,predicted_data)

    print("rmse:",rmse)
    print("mae:",mae)
    print("r2:",r2)

    # Log the metrics,parameters and model on mlflow
    mlflow.log_param("alpha",alpha)
    mlflow.log_param("l1_ratio",l1_ratio)

    mlflow.log_metric("rmse",rmse)
    mlflow.log_metric("mae",mae)
    mlflow.log_metric("r2",r2)

    # Register the model
    mlflow.sklearn.log_model(lr,"model",registered_model_name='ElasticNet')


In [17]:
train_model(0.3,0.2)

rmse: 0.8015619554750656
mae: 0.6146041041484648
r2: 0.18319008419651783


2025/01/01 23:51:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\igor\AppData\Local\Temp\tmpw53bsilw\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/01/01 23:51:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNet' already exists. Creating a new version of this model...
Created version '2' of model 'ElasticNet'.


In [18]:
train_model(0.8,0.6)

rmse: 0.8716969189431293
mae: 0.6497835908131276
r2: 0.033998432770415254


2025/01/01 23:52:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\igor\AppData\Local\Temp\tmpyy7izy1x\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/01/01 23:52:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticNet' already exists. Creating a new version of this model...
Created version '3' of model 'ElasticNet'.


In [19]:
# Create mlflowUI
!pip install pyngrok

In [23]:
from pyngrok import ngrok
# ngrok.kill()  # Kill any previous tunnels
auth_token="2r1qRGD8DuXNtAuDGIiKxQbOFzh_4wHo8MEftvS6DDFhpeshb"
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(addr='5000', proto='http')
print("Tracking URI:", ngrok_tunnel.public_url)


Tracking URI: https://3453-2603-6080-2f00-5888-e024-9fa1-b637-2e20.ngrok-free.app


In [24]:
import subprocess

# Run the MLflow UI command in the background and use http://localhost:5000 to access UI
subprocess.Popen(["mlflow", "ui"])


<Popen: returncode: None args: ['mlflow', 'ui']>

t=2025-01-02T00:55:00-0500 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=26c2410e69b0 clientid=3c02d25a40bbd71150c18a141cee87ff
t=2025-01-02T00:55:00-0500 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f432c84ada92 err="session closed"
t=2025-01-02T00:55:00-0500 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2025-01-02T01:11:48-0500 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=bfd4ab913dfd clientid=3c02d25a40bbd71150c18a141cee87ff
t=2025-01-02T01:11:48-0500 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=f432c84ada92 err="session closed"
t=2025-01-02T01:15:43-0500 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=acc4221c3153 client